In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import models.graph_gcn as mg
import models.adversarialNets as ma
import scipy.sparse
from utils import data_process, sparse
from utils import configs, metrics, feat2struct

Instructions for updating:
non-resource variables are not supported in the long term


Matplotlib created a temporary config/cache directory at /tmp/104723.tmpdir/matplotlib-i18amlwx because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

train_ratio = 120
dataset = configs.FILES.citeseer
lrate_gcn = configs.FILES.learning_rate
print("train_ratio:",train_ratio)

x, _, adj_norm, labels, train_indexes, test_indexes = data_process.load_data(dataset, str(train_ratio), 
                                                                                x_flag='feature')

node_num = adj_norm.shape[0]
label_num = labels.shape[1]

adj_norm_tuple = sparse.sparse_to_tuple(scipy.sparse.coo_matrix(adj_norm))
feat_x_nn_tuple = sparse.sparse_to_tuple(scipy.sparse.coo_matrix(x))

print(feat_x_nn_tuple)

# node-node network train and validate masks
nn_train_mask = np.zeros([node_num,])
nn_test_mask = np.zeros([node_num,])

for i in train_indexes:
    nn_train_mask[i] = 1
    nn_test_mask[i] = 0
    
for i in test_indexes:
    nn_test_mask[i] = 1
    
# TensorFlow placeholders
ph = {
      'adj_norm': tf.sparse_placeholder(tf.float32, name="adj_norm"),
      'x': tf.sparse_placeholder(tf.float32, name="features"),
      'labels': tf.placeholder(tf.float32, name="node_labels"),
      'mask': tf.placeholder(tf.int32, shape=(node_num,))
      }

placeholders = {
                'dropout_prob': tf.placeholder_with_default(0., shape=()),
                'num_features_nonzero': tf.placeholder(tf.int32)
                }


# the first layer
# feat_embeds, proj_X = feat2struct.BuildGraphStruct(out_size = 10,
#                                         dropout_prob = 0.3,
#                                         act = tf.nn.relu,
#                                         feat_dropout = True)(X = x.toarray())

t_model = mg.GraphConvLayer(input_dim=x.shape[-1],
                           output_dim=configs.FILES.h,
                           name='nn_fc1',
                           holders=placeholders,
                           act=tf.nn.relu,
                           dropout=True)

nn_fc1 = t_model(adj_norm=ph['adj_norm'],
                           x=ph['x'], sparse=True)
                            

# the second layer
nn_dl = mg.GraphConvLayer(input_dim=configs.FILES.h,
                           output_dim=label_num,
                           name='nn_dl',
                           holders=placeholders,
                           act=tf.nn.softmax,
                           dropout=True)(adj_norm=ph['adj_norm'],
                                           x=nn_fc1)  


def masked_sigmoid_softmax_cross_entropy(preds, labels, mask):
    """Sigmoid softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    for var in t_model.var.values():
        loss += configs.FILES.weight_decay * tf.nn.l2_loss(var)
    return tf.reduce_mean(loss)

def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    
    return tf.reduce_mean(accuracy_all), correct_prediction

# calculate the classification accuracy per classes   
def precision_per_class(preds, labels, mask):
    import heapq
    mask = mask.astype(int)
    labels = labels.astype(int)
    val_indexes = np.where(mask==1)[0]
    pred_true_labels = {}
    
    y_true = []
    y_pred = []    
    
    for i in val_indexes:
        pred_probs_i = preds[i]
        true_raw_i = labels[i]
        
        pred_label_i = heapq.nlargest(np.sum(true_raw_i),range(len(pred_probs_i)), 
                                      pred_probs_i.take)
        true_label_i = np.where(true_raw_i==1)[0]
        pred_true_labels[i] = (pred_label_i, true_label_i)
        
        y_true.append(true_label_i)
        y_pred.append(pred_label_i)
        
    accuracy_per_classes = metrics.evaluate(pred_true_labels)
    
    from sklearn.metrics import roc_curve, auc, confusion_matrix
    
    mat = confusion_matrix(y_true, y_pred)
    print(mat)        
        
    fpr = dict()
    tpr = dict()
    test_y = labels[val_indexes]
    test_pred = preds[val_indexes]
    fpr["micro"], tpr["micro"], _ = roc_curve(test_y.ravel(), test_pred.ravel())
    auc = auc(fpr["micro"], tpr["micro"])
    
    return accuracy_per_classes, auc
    
    from sklearn.metrics import roc_curve, auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    test_y = labels[val_indexes]
    test_pred = preds[val_indexes]
    fpr["micro"], tpr["micro"], _ = roc_curve(test_y.ravel(), test_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    print('micro_auc=',roc_auc["micro"])
    
    return accuracy_per_classes

with tf.name_scope('optimizer'):
    
    loss = masked_sigmoid_softmax_cross_entropy(preds=nn_dl, 
                                                labels=ph['labels'], mask=ph['mask'])
        
    accuracy, correct_prediction = masked_accuracy(preds=nn_dl, 
                               labels=ph['labels'], mask=ph['mask'])
    
    optimizer = tf.train.AdamOptimizer(learning_rate=lrate_gcn)
    opt_op = optimizer.minimize(loss)    

feed_dict_train = {ph['adj_norm']: adj_norm_tuple,
                      ph['x']: feat_x_nn_tuple,
#                     ph['labels']: labels,
                    ph['labels']: labels.toarray(),
                      ph['mask']: nn_train_mask,
                      placeholders['dropout_prob']: configs.FILES.dropout_prob,
                      placeholders['num_features_nonzero']: feat_x_nn_tuple[1].shape,
                      }
feed_dict_val = {ph['adj_norm']: adj_norm_tuple,
                    ph['x']: feat_x_nn_tuple,
#                     ph['labels']: labels,
                    ph['labels']: labels.toarray(),
                    ph['mask']: nn_test_mask,
                    placeholders['dropout_prob']: 0.,
                    placeholders['num_features_nonzero']: feat_x_nn_tuple[1].shape
                    }

sess = tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 400
save_every = 1    
    
t = time.time()
times = []
# Train model
for epoch in range(epochs):
    # Training node embedding
#     _, train_loss = sess.run(
#         (opt_op, loss), feed_dict=feed_dict_train)
    _, train_loss, train_acc, train_nn_dl = sess.run((opt_op, loss, accuracy, nn_dl), 
                                                              feed_dict=feed_dict_train)
    
    if epoch % save_every == 0:
        val_acc, test_nn_dl = sess.run((accuracy, nn_dl), feed_dict=feed_dict_val)
        
        print("Epoch:", '%04d' % (epoch + 1),
              "train_loss=", "{:.5f}".format(train_loss),
              "train_acc=", "{:.5f}".format(train_acc),
#               "train_auc", "{:.5f}".format(train_auc),
              "test_acc=", "{:.5f}".format(val_acc),
#               "test_auc", "{:.5f}".format(test_auc),
              "time=", "{:.5f}".format(time.time() - t))
    
    if epoch % 20 == 0:
        times.append(time.time() - t)


train_ratio: 120
label_distribution: [('0', 20), ('1', 20), ('2', 20), ('3', 20), ('4', 20), ('5', 20)]
balance_num: 20
(3312, 3703) (3312, 3312) (3312, 6) (120,) (1000,)
(array([[   0,    0],
       [   0,    1],
       [   0,    2],
       ...,
       [3311, 2536],
       [3311, 3073],
       [3311, 3589]], dtype=int32), array([1., 1., 1., ..., 1., 1., 1.]), (3312, 3703))
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 train_loss= 1.80574 train_acc= 0.14167 test_acc= 0.29700 time= 0.11727
Epoch: 0002 train_loss= 1.79971 train_acc= 0.27500 test_acc= 0.38500 time= 0.13963
Epoch: 0003 train_loss= 1.78854 train_acc= 0.41667 test_acc= 0.44000 time= 0.16062
Epoch: 0004 train_loss= 1.77934 train_acc= 0.55833 test_acc= 0.47200

Epoch: 0088 train_loss= 1.13261 train_acc= 1.00000 test_acc= 0.69000 time= 2.21283
Epoch: 0089 train_loss= 1.13215 train_acc= 0.99167 test_acc= 0.68800 time= 2.24242
Epoch: 0090 train_loss= 1.13564 train_acc= 0.99167 test_acc= 0.68500 time= 2.26655
Epoch: 0091 train_loss= 1.13741 train_acc= 0.99167 test_acc= 0.68200 time= 2.28798
Epoch: 0092 train_loss= 1.12409 train_acc= 0.99167 test_acc= 0.68100 time= 2.30960
Epoch: 0093 train_loss= 1.15007 train_acc= 0.98333 test_acc= 0.68200 time= 2.32973
Epoch: 0094 train_loss= 1.13440 train_acc= 1.00000 test_acc= 0.67900 time= 2.34924
Epoch: 0095 train_loss= 1.12964 train_acc= 0.99167 test_acc= 0.68000 time= 2.37052
Epoch: 0096 train_loss= 1.13279 train_acc= 0.99167 test_acc= 0.68000 time= 2.40048
Epoch: 0097 train_loss= 1.12720 train_acc= 1.00000 test_acc= 0.68300 time= 2.44010
Epoch: 0098 train_loss= 1.13096 train_acc= 0.99167 test_acc= 0.68200 time= 2.47974
Epoch: 0099 train_loss= 1.12908 train_acc= 0.99167 test_acc= 0.68300 time= 2.50289
Epoc

Epoch: 0189 train_loss= 1.10917 train_acc= 1.00000 test_acc= 0.68900 time= 4.56419
Epoch: 0190 train_loss= 1.10636 train_acc= 0.99167 test_acc= 0.68800 time= 4.59017
Epoch: 0191 train_loss= 1.10243 train_acc= 0.99167 test_acc= 0.68800 time= 4.61631
Epoch: 0192 train_loss= 1.10130 train_acc= 1.00000 test_acc= 0.68500 time= 4.63813
Epoch: 0193 train_loss= 1.10683 train_acc= 1.00000 test_acc= 0.68200 time= 4.65835
Epoch: 0194 train_loss= 1.10584 train_acc= 0.99167 test_acc= 0.68300 time= 4.67922
Epoch: 0195 train_loss= 1.10575 train_acc= 0.98333 test_acc= 0.68300 time= 4.70576
Epoch: 0196 train_loss= 1.10504 train_acc= 0.99167 test_acc= 0.68000 time= 4.74399
Epoch: 0197 train_loss= 1.10338 train_acc= 1.00000 test_acc= 0.68100 time= 4.78296
Epoch: 0198 train_loss= 1.10674 train_acc= 0.99167 test_acc= 0.68300 time= 4.81164
Epoch: 0199 train_loss= 1.10582 train_acc= 0.99167 test_acc= 0.68200 time= 4.85117
Epoch: 0200 train_loss= 1.10812 train_acc= 0.99167 test_acc= 0.68600 time= 4.88085
Epoc

Epoch: 0289 train_loss= 1.09061 train_acc= 1.00000 test_acc= 0.69200 time= 6.93489
Epoch: 0290 train_loss= 1.08968 train_acc= 1.00000 test_acc= 0.69200 time= 6.96305
Epoch: 0291 train_loss= 1.09107 train_acc= 0.99167 test_acc= 0.69100 time= 6.98440
Epoch: 0292 train_loss= 1.09069 train_acc= 0.99167 test_acc= 0.69100 time= 7.00534
Epoch: 0293 train_loss= 1.09183 train_acc= 0.99167 test_acc= 0.69100 time= 7.02589
Epoch: 0294 train_loss= 1.09858 train_acc= 1.00000 test_acc= 0.69000 time= 7.04826
Epoch: 0295 train_loss= 1.09614 train_acc= 0.99167 test_acc= 0.68900 time= 7.07013
Epoch: 0296 train_loss= 1.09332 train_acc= 1.00000 test_acc= 0.69000 time= 7.09195
Epoch: 0297 train_loss= 1.10074 train_acc= 0.98333 test_acc= 0.69000 time= 7.11347
Epoch: 0298 train_loss= 1.08793 train_acc= 1.00000 test_acc= 0.68900 time= 7.13384
Epoch: 0299 train_loss= 1.09194 train_acc= 1.00000 test_acc= 0.68900 time= 7.15352
Epoch: 0300 train_loss= 1.08932 train_acc= 0.99167 test_acc= 0.68900 time= 7.17386
Epoc

Epoch: 0390 train_loss= 1.08923 train_acc= 0.99167 test_acc= 0.69100 time= 9.28690
Epoch: 0391 train_loss= 1.08672 train_acc= 0.99167 test_acc= 0.69000 time= 9.31458
Epoch: 0392 train_loss= 1.08884 train_acc= 1.00000 test_acc= 0.68600 time= 9.34885
Epoch: 0393 train_loss= 1.08435 train_acc= 1.00000 test_acc= 0.68400 time= 9.37202
Epoch: 0394 train_loss= 1.08292 train_acc= 1.00000 test_acc= 0.68200 time= 9.39242
Epoch: 0395 train_loss= 1.08579 train_acc= 1.00000 test_acc= 0.68400 time= 9.41267
Epoch: 0396 train_loss= 1.08769 train_acc= 0.99167 test_acc= 0.68200 time= 9.44309
Epoch: 0397 train_loss= 1.08515 train_acc= 1.00000 test_acc= 0.68000 time= 9.47377
Epoch: 0398 train_loss= 1.09683 train_acc= 0.99167 test_acc= 0.68000 time= 9.50672
Epoch: 0399 train_loss= 1.08609 train_acc= 0.99167 test_acc= 0.67900 time= 9.55841
Epoch: 0400 train_loss= 1.08493 train_acc= 1.00000 test_acc= 0.68100 time= 9.59407
